In [257]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import csv

In [52]:
HEADERS = ({
    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64)AppleWebKit/537.36 (KHTML, like Gecko)Chrome/44.0.2403.157 Safari/537.36',
    'Accept-Language': 'en-US, en;q=0.5'
})

In [53]:
# page_number = 0
# status = 200
# i=0
# while True:
#     i+=1
#     url = f'https://www.themoviedb.org/movie?page={i}'
#     response = requests.get(url,headers=HEADERS)
#     print(i,end=' ')
#     if(response.status_code==200):
#         page_number+=1
#     else:
#         break
    
# print('total number of pages are :',page_number)


## commented this code because it was taking too long to run
## and i have to run whole file again and again after opening this

NOTE : Total number of pages are 500

In [237]:
movie_data_list = list()

In [238]:
def get_budget_and_revenue(page_soup):
    facts = page_soup.find('section',class_='facts left_column')
    if facts is not None:
        title,movie_budget = facts.find_all('bdi')[2].parent.parent.text.split()
        movie_budget = movie_budget.replace('$','')
        title,movie_revenue = facts.find_all('bdi')[3].parent.parent.text.split()
        movie_revenue = movie_revenue.replace('$','')
        movie_budget = float(movie_budget.replace(',','')) if movie_budget != '-' else np.nan
        movie_revenue = float(movie_revenue.replace(',','')) if movie_revenue != '-' else np.nan
        return movie_budget,movie_revenue
    return np.nan,np.nan

In [239]:
def get_title(page_soup):
    movie_title = page_soup.find('h2').find('a')
    return movie_title.text if movie_title is not None else 'unknown'

In [240]:
def get_user_score(page_soup):
    percentage = page_soup.find('div',class_='percent')
    if percentage is not None:
        percentage_string = percentage.find('span')['class'][1]
        index = percentage_string.index('r')
        user_score = percentage_string[index+1:] 
        try:
            user_score = eval(user_score)
            return user_score
        except:
            return np.nan
    return np.nan

In [241]:
def get_genres(page_soup):
    genres_list = 'unknown'
    genres = page_soup.find('span',class_='genres')
    return ','.join([genre.text for genre in genres.find_all('a')]) if genres is not None else 'unknown'

In [242]:
def get_release_date(page_soup):
    release_date_tag = page_soup.find('span',class_='release')
    if release_date_tag is not None:
        release_date,temp = release_date_tag.text.replace('\n','').split()
        year = pd.to_datetime(release_date).year
        month = pd.to_datetime(release_date).month
        day = pd.to_datetime(release_date).day
        weekday = pd.to_datetime(release_date).weekday()
        return year,month,day,weekday
    return np.nan,np.nan,np.nan,np.nan

In [243]:
def get_director_name(page_soup):
    director = page_soup.find('li',class_='profile')
    return director.find('a').text if director is not None else 'unknown'

In [244]:
def get_run_time(page_soup):
    run_time = page_soup.find('span',class_='runtime')
    if run_time is not None:
        time_string = run_time.text.replace('\n','').replace(' ','').replace('h',':').replace('m','')
        hours,minuts = time_string.split(':') if ':' in time_string else ('',time_string)
        hours = eval(hours) if hours != '' else 0
        minuts = eval(minuts) if minuts != '' else 0
        total_duration = (hours*60)+minuts
        return total_duration
    else:
        return np.nan

In [245]:
def get_content_score(page_soup):
    content_score_tag = page_soup.find('div',class_='content_score')
    return eval(content_score_tag.find('p').text) if content_score_tag is not None else np.nan

In [246]:
def get_movie_data(response):
    
    page_soup = BeautifulSoup(response.text,'html.parser')
    
    content_score = get_content_score(page_soup)
    
    budget,revenue = get_budget_and_revenue(page_soup)
    movie_title = get_title(page_soup)
    user_score = get_user_score(page_soup)
    genres = get_genres(page_soup)
    year,month,day,weekday = get_release_date(page_soup)
    director = get_director_name(page_soup)
    run_time = get_run_time(page_soup)
    return {
        'budget':budget,
        'revenue':revenue,
        'content_score':content_score,
        'user_score':user_score,
        'genres':genres,
        'director':director,
        'release_year':year,
        'release_month':month,
        'release_day':day,
        'release_weekday':weekday,
        'run_time (in minuts)':run_time,
        'title':movie_title
    }

In [247]:
def get_movies(movie_data):
    for movie in movie_data:    
        movie_page_url = f"https://www.themoviedb.org/{movie.find('a')['href']}-{movie.find('a')['title'].lower().replace(':','').replace(' ','-')}"
        print(movie_page_url)
        movie_page_response = requests.get(movie_page_url,headers=HEADERS)
        movie_info = get_movie_data(movie_page_response)
        movie_info['url'] = movie_page_url
        movie_data_list.append(movie_info)
        

In [248]:
def get_data(starting=1,ending=10):
    for i in range(starting,ending+1):
        print(f'page number {i}')
        url = f'https://www.themoviedb.org/movie?page={i}'
        response = requests.get(url,headers=HEADERS)
        page= BeautifulSoup(response.text, 'html.parser')
        movie_data = page.find_all('div',class_='card style_1')
        get_movies(movie_data = movie_data)
        print('\n')
    
    print("DONE")

In [249]:
get_data(starting=1,ending=100)

page number 1
https://www.themoviedb.org//movie/566525-shang-chi-and-the-legend-of-the-ten-rings
https://www.themoviedb.org//movie/370172-no-time-to-die
https://www.themoviedb.org//movie/580489-venom-let-there-be-carnage
https://www.themoviedb.org//movie/522402-finch
https://www.themoviedb.org//movie/763164-apex
https://www.themoviedb.org//movie/744275-after-we-fell
https://www.themoviedb.org//movie/438631-dune
https://www.themoviedb.org//movie/796499-army-of-thieves
https://www.themoviedb.org//movie/524434-eternals
https://www.themoviedb.org//movie/618162-the-harder-they-fall
https://www.themoviedb.org//movie/550988-free-guy
https://www.themoviedb.org//movie/885110-amina
https://www.themoviedb.org//movie/585245-clifford-the-big-red-dog
https://www.themoviedb.org//movie/843241-the-seven-deadly-sins-cursed-by-light
https://www.themoviedb.org//movie/335983-venom
https://www.themoviedb.org//movie/714968-plan-a
https://www.themoviedb.org//movie/871964-nobody-sleeps-in-the-woods-tonight-2
h

https://www.themoviedb.org//movie/674-harry-potter-and-the-goblet-of-fire
https://www.themoviedb.org//movie/12445-harry-potter-and-the-deathly-hallows-part-2
https://www.themoviedb.org//movie/375211-marvel-knights-eternals
https://www.themoviedb.org//movie/673-harry-potter-and-the-prisoner-of-azkaban
https://www.themoviedb.org//movie/317442-the-last-naruto-the-movie


page number 8
https://www.themoviedb.org//movie/24428-the-avengers
https://www.themoviedb.org//movie/559-spider-man-3
https://www.themoviedb.org//movie/800497-werewolves-within
https://www.themoviedb.org//movie/550205-wish-dragon
https://www.themoviedb.org//movie/508442-soul
https://www.themoviedb.org//movie/678580-el-mesero
https://www.themoviedb.org//movie/12444-harry-potter-and-the-deathly-hallows-part-1
https://www.themoviedb.org//movie/617502-jolt
https://www.themoviedb.org//movie/482321-ron's-gone-wrong
https://www.themoviedb.org//movie/719088-yes,-no,-or-maybe-half?
https://www.themoviedb.org//movie/839100-bright-s

https://www.themoviedb.org//movie/44571-the-king-of-fighters
https://www.themoviedb.org//movie/431075-polaroid
https://www.themoviedb.org//movie/800823-the-kids-are-alright-destination-asturias
https://www.themoviedb.org//movie/99861-avengers-age-of-ultron
https://www.themoviedb.org//movie/786705-confessions-of-an-invisible-girl
https://www.themoviedb.org//movie/425-ice-age
https://www.themoviedb.org//movie/483898-50-greatest-harry-potter-moments
https://www.themoviedb.org//movie/723072-host
https://www.themoviedb.org//movie/214756-ted-2
https://www.themoviedb.org//movie/284052-doctor-strange
https://www.themoviedb.org//movie/91314-transformers-age-of-extinction
https://www.themoviedb.org//movie/633515-the-forgotten-battle
https://www.themoviedb.org//movie/18785-the-hangover
https://www.themoviedb.org//movie/27205-inception


page number 15
https://www.themoviedb.org//movie/647302-benny-loves-you
https://www.themoviedb.org//movie/360920-the-grinch
https://www.themoviedb.org//movie/6768

https://www.themoviedb.org//movie/198184-chappie
https://www.themoviedb.org//movie/595280-fever-dream
https://www.themoviedb.org//movie/663558-new-gods-nezha-reborn
https://www.themoviedb.org//movie/786300-tentacles
https://www.themoviedb.org//movie/579047-reminiscence
https://www.themoviedb.org//movie/615677-we-can-be-heroes
https://www.themoviedb.org//movie/577922-tenet
https://www.themoviedb.org//movie/486589-red-shoes-and-the-seven-dwarfs
https://www.themoviedb.org//movie/299537-captain-marvel
https://www.themoviedb.org//movie/223702-sausage-party
https://www.themoviedb.org//movie/398818-call-me-by-your-name
https://www.themoviedb.org//movie/617708-la-llorona
https://www.themoviedb.org//movie/4247-scary-movie
https://www.themoviedb.org//movie/420818-the-lion-king
https://www.themoviedb.org//movie/793002-dreamcatcher
https://www.themoviedb.org//movie/632632-given


page number 22
https://www.themoviedb.org//movie/38365-grown-ups
https://www.themoviedb.org//movie/254128-san-andreas
h



page number 28
https://www.themoviedb.org//movie/689249-money-heist-the-phenomenon
https://www.themoviedb.org//movie/50620-the-twilight-saga-breaking-dawn---part-2
https://www.themoviedb.org//movie/285-pirates-of-the-caribbean-at-world's-end
https://www.themoviedb.org//movie/667520-a-whisker-away
https://www.themoviedb.org//movie/55846-blitz
https://www.themoviedb.org//movie/72105-ted
https://www.themoviedb.org//movie/23629-sucker-punch
https://www.themoviedb.org//movie/760883-blood-red-sky
https://www.themoviedb.org//movie/39513-paul
https://www.themoviedb.org//movie/109418-grown-ups-2
https://www.themoviedb.org//movie/819413-august-never-ends
https://www.themoviedb.org//movie/396535-train-to-busan
https://www.themoviedb.org//movie/662546-godmothered
https://www.themoviedb.org//movie/681260-maya-the-bee-the-golden-orb
https://www.themoviedb.org//movie/16996-17-again
https://www.themoviedb.org//movie/749618-xico's-journey
https://www.themoviedb.org//movie/23823-wrong-turn-3-left-for-

...



page number 69
https://www.themoviedb.org//movie/845222-kingdom-ashin-of-the-north
https://www.themoviedb.org//movie/75780-jack-reacher
https://www.themoviedb.org//movie/10679-iron-sky
https://www.themoviedb.org//movie/10867-malena
https://www.themoviedb.org//movie/15016-barbie-of-swan-lake
https://www.themoviedb.org//movie/13285-barbie-fairytopia-mermaidia
https://www.themoviedb.org//movie/9323-ghost-in-the-shell
https://www.themoviedb.org//movie/75258-secret-of-the-wings
https://www.themoviedb.org//movie/16366-joseph-king-of-dreams
https://www.themoviedb.org//movie/13955-beethoven's-big-break
https://www.themoviedb.org//movie/564251-kakegurui-the-movie
https://www.themoviedb.org//movie/43947-i-spit-on-your-grave
https://www.themoviedb.org//movie/330459-rogue-one-a-star-wars-story
https://www.themoviedb.org//movie/282069-parasyte-part-1
https://www.themoviedb.org//movie/552687-wotakoi-love-is-hard-for-otaku
https://www.themoviedb.org//movie/663-saw-iv
https://www.themoviedb.org//mov

https://www.themoviedb.org//movie/9757-norbit
https://www.themoviedb.org//movie/25259-seven-minutes
https://www.themoviedb.org//movie/275619-age-of-tomorrow


page number 76
https://www.themoviedb.org//movie/739936-french-lolita
https://www.themoviedb.org//movie/840427-the-father-who-moves-mountains
https://www.themoviedb.org//movie/41446-scream-4
https://www.themoviedb.org//movie/590223-love-and-monsters
https://www.themoviedb.org//movie/315465-the-boy-and-the-beast
https://www.themoviedb.org//movie/79218-ice-age-a-mammoth-christmas
https://www.themoviedb.org//movie/824920-stop
https://www.themoviedb.org//movie/254-king-kong
https://www.themoviedb.org//movie/624956-sonic
https://www.themoviedb.org//movie/474392-samson
https://www.themoviedb.org//movie/448441-sekaiichi-hatsukoi-movie
https://www.themoviedb.org//movie/228161-home
https://www.themoviedb.org//movie/14128-cinderella-ii-dreams-come-true
https://www.themoviedb.org//movie/5175-rush-hour-2
https://www.themoviedb.org//movie/195

https://www.themoviedb.org//movie/256274-as-above,-so-below
https://www.themoviedb.org//movie/4232-scream
https://www.themoviedb.org//movie/127372-emperor
https://www.themoviedb.org//movie/571648-beasts-clawing-at-straws
https://www.themoviedb.org//movie/18937-quest-for-camelot
https://www.themoviedb.org//movie/716258-black-box
https://www.themoviedb.org//movie/16907-naruto-the-movie-ninja-clash-in-the-land-of-snow
https://www.themoviedb.org//movie/98-gladiator


page number 83
https://www.themoviedb.org//movie/10140-the-chronicles-of-narnia-the-voyage-of-the-dawn-treader
https://www.themoviedb.org//movie/7443-chicken-run
https://www.themoviedb.org//movie/79224-war-of-the-arrows
https://www.themoviedb.org//movie/593655-castle-freak
https://www.themoviedb.org//movie/347126-god's-not-dead-2
https://www.themoviedb.org//movie/381284-hidden-figures
https://www.themoviedb.org//movie/13459-barbie-in-'a-christmas-carol'
https://www.themoviedb.org//movie/774372-ariana-grande-excuse-me,-i-love-y

https://www.themoviedb.org//movie/127521-6-bullets
https://www.themoviedb.org//movie/254470-pitch-perfect-2
https://www.themoviedb.org//movie/6537-the-orphanage
https://www.themoviedb.org//movie/666624-security
https://www.themoviedb.org//movie/283984-fate/stay-night-heaven's-feel-i.-presage-flower
https://www.themoviedb.org//movie/10285-jason-goes-to-hell-the-final-friday
https://www.themoviedb.org//movie/453362-los-perros
https://www.themoviedb.org//movie/23566-barbie-and-the-three-musketeers
https://www.themoviedb.org//movie/698916-sunny-side-battle!
https://www.themoviedb.org//movie/602-independence-day
https://www.themoviedb.org//movie/46529-i-am-number-four


page number 90
https://www.themoviedb.org//movie/8273-american-wedding
https://www.themoviedb.org//movie/171424-the-green-inferno
https://www.themoviedb.org//movie/239523-hunter-x-hunter-the-last-mission
https://www.themoviedb.org//movie/467988-camp-cool-kids
https://www.themoviedb.org//movie/153158-underdogs
https://www.the

https://www.themoviedb.org//movie/72358-a-thousand-words
https://www.themoviedb.org//movie/3980-critters
https://www.themoviedb.org//movie/613911-bliss
https://www.themoviedb.org//movie/413518-pinocchio
https://www.themoviedb.org//movie/9912-the-world's-fastest-indian
https://www.themoviedb.org//movie/116946-the-wailer
https://www.themoviedb.org//movie/27579-the-american
https://www.themoviedb.org//movie/175112-tinker-bell-and-the-pirate-fairy
https://www.themoviedb.org//movie/6114-bram-stoker's-dracula
https://www.themoviedb.org//movie/10281-friday-the-13th-part-vii-the-new-blood
https://www.themoviedb.org//movie/9279-jingle-all-the-way
https://www.themoviedb.org//movie/877372-a-world-without
https://www.themoviedb.org//movie/294652-son-of-a-gun


page number 97
https://www.themoviedb.org//movie/333596-woodlawn
https://www.themoviedb.org//movie/7191-cloverfield
https://www.themoviedb.org//movie/16538-i-love-you,-man
https://www.themoviedb.org//movie/738542-the-haunting-of-margam-castl

In [252]:
get_data(starting=101,ending=200)

page number 101
https://www.themoviedb.org//movie/608195-ghosts-of-war
https://www.themoviedb.org//movie/482408-harry-potter-a-history-of-magic
https://www.themoviedb.org//movie/429415-extinction
https://www.themoviedb.org//movie/407655-a-cinderella-story-if-the-shoe-fits
https://www.themoviedb.org//movie/745881-the-medium
https://www.themoviedb.org//movie/137227-doraemon-nobita-and-the-new-steel-troops-~winged-angels~
https://www.themoviedb.org//movie/74997-the-human-centipede-2-(full-sequence)
https://www.themoviedb.org//movie/316029-the-greatest-showman
https://www.themoviedb.org//movie/68812-the-iceman
https://www.themoviedb.org//movie/471506-greta
https://www.themoviedb.org//movie/625450-tall-girl
https://www.themoviedb.org//movie/169-predator-2
https://www.themoviedb.org//movie/11247-a-cinderella-story
https://www.themoviedb.org//movie/1417-pan's-labyrinth
https://www.themoviedb.org//movie/501170-doctor-sleep
https://www.themoviedb.org//movie/181283-child-44
https://www.themovied

https://www.themoviedb.org//movie/523638-prisoners-of-the-ghostland
https://www.themoviedb.org//movie/242546-bayonetta-bloody-fate
https://www.themoviedb.org//movie/954-mission-impossible
https://www.themoviedb.org//movie/179053-app
https://www.themoviedb.org//movie/257211-the-intern
https://www.themoviedb.org//movie/77459-a-monster-in-paris


page number 108
https://www.themoviedb.org//movie/606856-togo
https://www.themoviedb.org//movie/307479-88
https://www.themoviedb.org//movie/77499-barbie-a-perfect-christmas
https://www.themoviedb.org//movie/83542-cloud-atlas
https://www.themoviedb.org//movie/732670-the-lego-star-wars-holiday-special
https://www.themoviedb.org//movie/672741-till-death
https://www.themoviedb.org//movie/453395-doctor-strange-in-the-multiverse-of-madness
https://www.themoviedb.org//movie/699570-boruto-jump-festa-special
https://www.themoviedb.org//movie/280217-the-lego-movie-2-the-second-part
https://www.themoviedb.org//movie/785534-paper-lives
https://www.themoviedb

https://www.themoviedb.org//movie/494656-la-leyenda-del-charro-negro
https://www.themoviedb.org//movie/205724-welcome-to-the-jungle
https://www.themoviedb.org//movie/1573-die-hard-2
https://www.themoviedb.org//movie/392681-museum
https://www.themoviedb.org//movie/75638-red-lights
https://www.themoviedb.org//movie/9603-clueless
https://www.themoviedb.org//movie/332567-the-shallows
https://www.themoviedb.org//movie/11453-deuce-bigalow-european-gigolo
https://www.themoviedb.org//movie/2123-me,-myself-&-irene


page number 115
https://www.themoviedb.org//movie/142061-batman-the-dark-knight-returns,-part-2
https://www.themoviedb.org//movie/496704-tremors-a-cold-day-in-hell
https://www.themoviedb.org//movie/673319-happily
https://www.themoviedb.org//movie/12222-horton-hears-a-who!
https://www.themoviedb.org//movie/337719-possessed
https://www.themoviedb.org//movie/832359-aew-full-gear-2021
https://www.themoviedb.org//movie/586097-robert-reborn
https://www.themoviedb.org//movie/353616-pitch-p

https://www.themoviedb.org//movie/13649-high-school-musical-2
https://www.themoviedb.org//movie/397837-before-i-fall
https://www.themoviedb.org//movie/621-grease
https://www.themoviedb.org//movie/1255-the-host
https://www.themoviedb.org//movie/299963-dragon-ball-z-gather-together!-goku's-world
https://www.themoviedb.org//movie/10515-castle-in-the-sky
https://www.themoviedb.org//movie/78-blade-runner
https://www.themoviedb.org//movie/4234-scream-3
https://www.themoviedb.org//movie/144336-sabotage


page number 122
https://www.themoviedb.org//movie/204342-hk-forbidden-super-hero
https://www.themoviedb.org//movie/11887-high-school-musical-3-senior-year
https://www.themoviedb.org//movie/525460-welcome-home
https://www.themoviedb.org//movie/44282-plan-b
https://www.themoviedb.org//movie/1572-die-hard-with-a-vengeance
https://www.themoviedb.org//movie/221731-rurouni-kenshin-part-ii-kyoto-inferno
https://www.themoviedb.org//movie/108726-a-common-man
https://www.themoviedb.org//movie/599399-af

https://www.themoviedb.org//movie/120475-dragon-ball-episode-of-bardock
https://www.themoviedb.org//movie/520663-the-woman-in-the-window
https://www.themoviedb.org//movie/82881-tangled-ever-after
https://www.themoviedb.org//movie/645488-aquaman-heroines-of-atlantis
https://www.themoviedb.org//movie/458723-us
https://www.themoviedb.org//movie/121734-tad,-the-lost-explorer
https://www.themoviedb.org//movie/27582-the-mechanic
https://www.themoviedb.org//movie/578189-black-and-blue
https://www.themoviedb.org//movie/242-the-godfather-part-iii


page number 129
https://www.themoviedb.org//movie/451644-dragonheart-battle-for-the-heartfire
https://www.themoviedb.org//movie/295011-patient-zero
https://www.themoviedb.org//movie/9320-the-avengers
https://www.themoviedb.org//movie/579583-the-king-of-staten-island
https://www.themoviedb.org//movie/1452-superman-returns
https://www.themoviedb.org//movie/24100-the-little-vampire
https://www.themoviedb.org//movie/51482-death-note-relight-1-visions-of-

...

https://www.themoviedb.org//movie/47931-elite-squad-the-enemy-within
https://www.themoviedb.org//movie/508664-danger-close-the-battle-of-long-tan
https://www.themoviedb.org//movie/311-once-upon-a-time-in-america


page number 170
https://www.themoviedb.org//movie/537059-justice-league-vs.-the-fatal-five
https://www.themoviedb.org//movie/439213-more-than-miyagi-the-pat-morita-story
https://www.themoviedb.org//movie/10073-date-movie
https://www.themoviedb.org//movie/526051-look-away
https://www.themoviedb.org//movie/310119-last-days-in-the-desert
https://www.themoviedb.org//movie/829729-potter's-ground
https://www.themoviedb.org//movie/13417-kronk's-new-groove
https://www.themoviedb.org//movie/567797-the-dry
https://www.themoviedb.org//movie/2024-the-patriot
https://www.themoviedb.org//movie/310307-the-founder
https://www.themoviedb.org//movie/893085-like-the-wind
https://www.themoviedb.org//movie/605116-project-power
https://www.themoviedb.org//movie/437557-blockers
https://www.themovie

https://www.themoviedb.org//movie/836486-jfk-revisited-through-the-looking-glass
https://www.themoviedb.org//movie/11618-the-haunting
https://www.themoviedb.org//movie/420426-bleach
https://www.themoviedb.org//movie/211755-hunter-x-hunter-phantom-rouge
https://www.themoviedb.org//movie/76025-shame


page number 177
https://www.themoviedb.org//movie/50647-wanderlust
https://www.themoviedb.org//movie/469916-downrange
https://www.themoviedb.org//movie/783546-hideout
https://www.themoviedb.org//movie/5549-robocop-2
https://www.themoviedb.org//movie/62395-the-miracle-of-marcelino
https://www.themoviedb.org//movie/770-gone-with-the-wind
https://www.themoviedb.org//movie/431530-a-bad-moms-christmas
https://www.themoviedb.org//movie/72570-the-vow
https://www.themoviedb.org//movie/846-the-x-files
https://www.themoviedb.org//movie/520901-doom-annihilation
https://www.themoviedb.org//movie/258099-just-before-i-go
https://www.themoviedb.org//movie/83899-you're-next
https://www.themoviedb.org//movi

https://www.themoviedb.org//movie/9276-the-faculty
https://www.themoviedb.org//movie/10951-gorgeous
https://www.themoviedb.org//movie/9928-robots
https://www.themoviedb.org//movie/433128-ajin-demi-human
https://www.themoviedb.org//movie/790496-bad-luck-banging-or-loony-porn
https://www.themoviedb.org//movie/795-city-of-angels
https://www.themoviedb.org//movie/796989-death-saved-my-life


page number 184
https://www.themoviedb.org//movie/49049-dredd
https://www.themoviedb.org//movie/16052-selena
https://www.themoviedb.org//movie/114372-fist-of-the-north-star-legend-of-raoh---chapter-of-fierce-fight
https://www.themoviedb.org//movie/306819-the-danish-girl
https://www.themoviedb.org//movie/4638-hot-fuzz
https://www.themoviedb.org//movie/10366-universal-soldier-the-return
https://www.themoviedb.org//movie/814-an-american-werewolf-in-london
https://www.themoviedb.org//movie/476273-mater
https://www.themoviedb.org//movie/11474-the-warriors
https://www.themoviedb.org//movie/327331-the-dirt
ht

https://www.themoviedb.org//movie/286548-tekken-kazuya's-revenge
https://www.themoviedb.org//movie/500921-between-worlds
https://www.themoviedb.org//movie/106747-machete-kills
https://www.themoviedb.org//movie/560044-the-willoughbys
https://www.themoviedb.org//movie/288036-sleeping-with-other-people
https://www.themoviedb.org//movie/345235-monk-comes-down-the-mountain
https://www.themoviedb.org//movie/392044-murder-on-the-orient-express
https://www.themoviedb.org//movie/877813-kumar


page number 191
https://www.themoviedb.org//movie/629017-run-hide-fight
https://www.themoviedb.org//movie/554241-playing-with-fire
https://www.themoviedb.org//movie/166076-superman-unbound
https://www.themoviedb.org//movie/40205-16-wishes
https://www.themoviedb.org//movie/610892-violet-evergarden-eternity-and-the-auto-memory-doll
https://www.themoviedb.org//movie/426865-ufo
https://www.themoviedb.org//movie/76122-marvel-one-shot-the-consultant
https://www.themoviedb.org//movie/572609-school-live!
https://

https://www.themoviedb.org//movie/13888-return-to-the-blue-lagoon
https://www.themoviedb.org//movie/1977-the-grudge-3
https://www.themoviedb.org//movie/11418-national-lampoon's-european-vacation
https://www.themoviedb.org//movie/767499-the-manor
https://www.themoviedb.org//movie/10172-are-we-done-yet?
https://www.themoviedb.org//movie/156708-afternoon-delight
https://www.themoviedb.org//movie/102629-el-gringo
https://www.themoviedb.org//movie/401773-if-it-bleeds-we-can-kill-it-the-making-of-'predator'
https://www.themoviedb.org//movie/11636-new-police-story
https://www.themoviedb.org//movie/326947-the-maid
https://www.themoviedb.org//movie/486131-shaft
https://www.themoviedb.org//movie/582570-sextuplets


page number 198
https://www.themoviedb.org//movie/742391-final-set
https://www.themoviedb.org//movie/369885-allied
https://www.themoviedb.org//movie/3049-ace-ventura-pet-detective
https://www.themoviedb.org//movie/473637-invisible
https://www.themoviedb.org//movie/11329-runaway-jury
h

In [253]:
get_data(starting=201,ending=300)

page number 201
https://www.themoviedb.org//movie/723895-dragons-rescue-riders-secrets-of-the-songwing
https://www.themoviedb.org//movie/318850-the-young-messiah
https://www.themoviedb.org//movie/1911-the-13th-warrior
https://www.themoviedb.org//movie/354278-che-dottoressa-ragazzi!
https://www.themoviedb.org//movie/441282-night-hunter
https://www.themoviedb.org//movie/25602-ninja
https://www.themoviedb.org//movie/184341-hands-of-stone
https://www.themoviedb.org//movie/5550-robocop-3
https://www.themoviedb.org//movie/16281-creepshow
https://www.themoviedb.org//movie/523777-next-gen
https://www.themoviedb.org//movie/182026-stranger-things
https://www.themoviedb.org//movie/13683-tarzan-&-jane
https://www.themoviedb.org//movie/65055-shark-night-3d
https://www.themoviedb.org//movie/4226-shriek-if-you-know-what-i-did-last-friday-the-thirteenth
https://www.themoviedb.org//movie/470114-24-hours-to-live
https://www.themoviedb.org//movie/79516-tekken-blood-vengeance
https://www.themoviedb.org//m



page number 208
https://www.themoviedb.org//movie/836487-alan-saldaña-locked-up
https://www.themoviedb.org//movie/313922-green-room
https://www.themoviedb.org//movie/483104-a-christmas-prince
https://www.themoviedb.org//movie/870606-pokémon-apokélypse
https://www.themoviedb.org//movie/10160-a-nightmare-on-elm-street-the-dream-child
https://www.themoviedb.org//movie/343203-emanuelle-e-lolita
https://www.themoviedb.org//movie/20235-belle's-magical-world
https://www.themoviedb.org//movie/10734-escape-from-alcatraz
https://www.themoviedb.org//movie/833404-bloody-oranges
https://www.themoviedb.org//movie/184346-deliver-us-from-evil
https://www.themoviedb.org//movie/323676-the-boss
https://www.themoviedb.org//movie/752505-the-old-ways
https://www.themoviedb.org//movie/504997-lego-dc-comics-super-heroes-the-flash
https://www.themoviedb.org//movie/10578-dracula-ii-ascension
https://www.themoviedb.org//movie/232572-a-walk-in-the-woods
https://www.themoviedb.org//movie/298312-the-visit
https:/

https://www.themoviedb.org//movie/2085-romeo-must-die
https://www.themoviedb.org//movie/307663-vice


page number 215
https://www.themoviedb.org//movie/698-moonraker
https://www.themoviedb.org//movie/686245-survive-the-night
https://www.themoviedb.org//movie/686-contact
https://www.themoviedb.org//movie/13313-beauty-and-the-beast-the-enchanted-christmas
https://www.themoviedb.org//movie/339380-on-the-basis-of-sex
https://www.themoviedb.org//movie/10027-unleashed
https://www.themoviedb.org//movie/145135-dark-skies
https://www.themoviedb.org//movie/297291-the-scorpion-king-4-quest-for-power
https://www.themoviedb.org//movie/254904-the-november-man
https://www.themoviedb.org//movie/610394-the-haunting-of-la-llorona
https://www.themoviedb.org//movie/19585-g-force
https://www.themoviedb.org//movie/52274-julia's-eyes
https://www.themoviedb.org//movie/811769-the-first-wave
https://www.themoviedb.org//movie/264873-graf-porno-und-die-liebesdurstigen-töchter
https://www.themoviedb.org//movie/945

https://www.themoviedb.org//movie/11230-drunken-master
https://www.themoviedb.org//movie/36218-pokémon-jirachi-wish-maker
https://www.themoviedb.org//movie/77016-end-of-watch
https://www.themoviedb.org//movie/9871-ginger-snaps
https://www.themoviedb.org//movie/10529-outlander
https://www.themoviedb.org//movie/8386-how-high
https://www.themoviedb.org//movie/128876-tower-block
https://www.themoviedb.org//movie/41216-legend-of-the-guardians-the-owls-of-ga'hoole
https://www.themoviedb.org//movie/449563-isn't-it-romantic
https://www.themoviedb.org//movie/9476-a-knight's-tale
https://www.themoviedb.org//movie/435107-a-royal-winter
https://www.themoviedb.org//movie/47292-pokémon-giratina-and-the-sky-warrior
https://www.themoviedb.org//movie/449459-the-vanishing
https://www.themoviedb.org//movie/515333-amateur
https://www.themoviedb.org//movie/10756-the-haunted-mansion
https://www.themoviedb.org//movie/50456-hanna
https://www.themoviedb.org//movie/127867-the-brass-teapot
https://www.themoviedb

https://www.themoviedb.org//movie/260234-monster-high-frights,-camera,-action!
https://www.themoviedb.org//movie/171274-inherent-vice
https://www.themoviedb.org//movie/235271-addicted
https://www.themoviedb.org//movie/230896-iron-man-&-hulk-heroes-united
https://www.themoviedb.org//movie/677115-female-urologists-2
https://www.themoviedb.org//movie/9056-police-story
https://www.themoviedb.org//movie/9920-ultraviolet
https://www.themoviedb.org//movie/555974-brahms-the-boy-ii
https://www.themoviedb.org//movie/156700-the-kings-of-summer
https://www.themoviedb.org//movie/881460-planet-dune
https://www.themoviedb.org//movie/714375-fatal-affair
https://www.themoviedb.org//movie/2023-hidalgo
https://www.themoviedb.org//movie/431261-american-wrestler-the-wizard
https://www.themoviedb.org//movie/88042-parental-guidance
https://www.themoviedb.org//movie/12094-jackass-number-two
https://www.themoviedb.org//movie/504-monster
https://www.themoviedb.org//movie/258230-a-monster-calls


page number 230

...

https://www.themoviedb.org//movie/1817-phone-booth
https://www.themoviedb.org//movie/750253-my-son
https://www.themoviedb.org//movie/188403-doraemon-nobita's-dorabian-nights
https://www.themoviedb.org//movie/883827-elles-dansent
https://www.themoviedb.org//movie/506072-prospect
https://www.themoviedb.org//movie/34806-the-back-up-plan
https://www.themoviedb.org//movie/8489-ali
https://www.themoviedb.org//movie/664-twister
https://www.themoviedb.org//movie/10802-showgirls
https://www.themoviedb.org//movie/10246-porky's
https://www.themoviedb.org//movie/25748-children-of-the-corn-ii-the-final-sacrifice
https://www.themoviedb.org//movie/36401-pornography
https://www.themoviedb.org//movie/16617-mindhunters
https://www.themoviedb.org//movie/474498-mariah-carey's-all-i-want-for-christmas-is-you
https://www.themoviedb.org//movie/268092-transformers-prime-beast-hunters-predacons-rising
https://www.themoviedb.org//movie/893030-children-of-the-mist


page number 272
https://www.themoviedb.org//mo

https://www.themoviedb.org//movie/81390-a-lonely-place-to-die
https://www.themoviedb.org//movie/8764-top-secret!
https://www.themoviedb.org//movie/8741-the-thin-red-line
https://www.themoviedb.org//movie/535167-the-wandering-earth
https://www.themoviedb.org//movie/13812-quarantine
https://www.themoviedb.org//movie/617-wild-things
https://www.themoviedb.org//movie/324560-brimstone
https://www.themoviedb.org//movie/22582-tom-and-jerry-the-movie
https://www.themoviedb.org//movie/3489-88-minutes
https://www.themoviedb.org//movie/368940-robinson-crusoe
https://www.themoviedb.org//movie/130150-labor-day
https://www.themoviedb.org//movie/79548-gone
https://www.themoviedb.org//movie/362826-wildling
https://www.themoviedb.org//movie/612706-work-it


page number 279
https://www.themoviedb.org//movie/10851-revolver
https://www.themoviedb.org//movie/455957-domino
https://www.themoviedb.org//movie/11864-enemy-mine
https://www.themoviedb.org//movie/210479-locke
https://www.themoviedb.org//movie/1337

https://www.themoviedb.org//movie/466550-drive
https://www.themoviedb.org//movie/12160-wyatt-earp
https://www.themoviedb.org//movie/659991-made-in-italy
https://www.themoviedb.org//movie/612152-secret-obsession
https://www.themoviedb.org//movie/322456-lego-dc-comics-super-heroes-justice-league-vs.-bizarro-league
https://www.themoviedb.org//movie/244506-camp-x-ray
https://www.themoviedb.org//movie/787723-13-minutes
https://www.themoviedb.org//movie/300386-justice-league-secret-origins
https://www.themoviedb.org//movie/302349-iron-sky-the-coming-race
https://www.themoviedb.org//movie/2609-planes,-trains-and-automobiles
https://www.themoviedb.org//movie/396461-under-the-silver-lake
https://www.themoviedb.org//movie/519465-queen-of-hearts
https://www.themoviedb.org//movie/597856-critters-attack!
https://www.themoviedb.org//movie/364433-they-call-me-jeeg
https://www.themoviedb.org//movie/9671-crocodile-dundee


page number 286
https://www.themoviedb.org//movie/892865-ultraviolette-and-the-b

https://www.themoviedb.org//movie/9913-the-skeleton-key
https://www.themoviedb.org//movie/877880-叶问宗师觉醒
https://www.themoviedb.org//movie/9788-accepted
https://www.themoviedb.org//movie/575776-saint-maud
https://www.themoviedb.org//movie/1272-sunshine
https://www.themoviedb.org//movie/574982-the-blackout
https://www.themoviedb.org//movie/813-airplane!
https://www.themoviedb.org//movie/926-galaxy-quest
https://www.themoviedb.org//movie/444539-the-bookshop
https://www.themoviedb.org//movie/3525-working-girl
https://www.themoviedb.org//movie/9569-hard-to-kill
https://www.themoviedb.org//movie/394117-the-florida-project
https://www.themoviedb.org//movie/21301-barbershop-2-back-in-business
https://www.themoviedb.org//movie/10944-in-the-cut
https://www.themoviedb.org//movie/254320-the-lobster


page number 293
https://www.themoviedb.org//movie/437586-mid90s
https://www.themoviedb.org//movie/418680-goodbye-christopher-robin
https://www.themoviedb.org//movie/23706-all-about-steve
https://www.t

https://www.themoviedb.org//movie/279547-cannibal-lolita
https://www.themoviedb.org//movie/11208-wicker-park
https://www.themoviedb.org//movie/10780-overboard
https://www.themoviedb.org//movie/8054-the-imaginarium-of-doctor-parnassus
https://www.themoviedb.org//movie/1955-the-elephant-man
https://www.themoviedb.org//movie/371638-the-disaster-artist
https://www.themoviedb.org//movie/247-the-killing
https://www.themoviedb.org//movie/10072-a-nightmare-on-elm-street-3-dream-warriors
https://www.themoviedb.org//movie/316152-free-state-of-jones
https://www.themoviedb.org//movie/424121-apostle
https://www.themoviedb.org//movie/661950-alone
https://www.themoviedb.org//movie/891443-resources
https://www.themoviedb.org//movie/433627-7-days-in-entebbe
https://www.themoviedb.org//movie/11449-the-amityville-horror


page number 300
https://www.themoviedb.org//movie/711705-they-reach
https://www.themoviedb.org//movie/461928-anna-and-the-apocalypse
https://www.themoviedb.org//movie/100271-a-letter-to

In [254]:
get_data(starting=301,ending=400)

page number 301
https://www.themoviedb.org//movie/332721-dad's-army
https://www.themoviedb.org//movie/9398-zoolander
https://www.themoviedb.org//movie/49948-fantasia-2000
https://www.themoviedb.org//movie/11846-father-of-the-bride
https://www.themoviedb.org//movie/11870-big-fat-liar
https://www.themoviedb.org//movie/294963-bone-tomahawk
https://www.themoviedb.org//movie/293452-the-disappointments-room
https://www.themoviedb.org//movie/996-double-indemnity
https://www.themoviedb.org//movie/11075-audition
https://www.themoviedb.org//movie/11931-in-her-shoes
https://www.themoviedb.org//movie/34803-death-at-a-funeral
https://www.themoviedb.org//movie/516850-the-mermaid-lake-of-the-dead
https://www.themoviedb.org//movie/11838-ju-on-the-grudge
https://www.themoviedb.org//movie/508947-turning-red
https://www.themoviedb.org//movie/12700-dominion-prequel-to-the-exorcist
https://www.themoviedb.org//movie/860-wargames
https://www.themoviedb.org//movie/477044-bergman-island
https://www.themoviedb.



page number 308
https://www.themoviedb.org//movie/360404-monster-high-boo-york,-boo-york
https://www.themoviedb.org//movie/93394-elevator
https://www.themoviedb.org//movie/590387-brian-wilson-long-promised-road
https://www.themoviedb.org//movie/16390-scooby-doo!-and-the-samurai-sword
https://www.themoviedb.org//movie/271709-new-initial-d-the-movie---legend-1-awakening
https://www.themoviedb.org//movie/60062-judy-moody-and-the-not-bummer-summer
https://www.themoviedb.org//movie/595723-franco-escamilla-bienvenido-al-mundo
https://www.themoviedb.org//movie/8408-day-of-the-dead
https://www.themoviedb.org//movie/938-for-a-few-dollars-more
https://www.themoviedb.org//movie/489430-terrified
https://www.themoviedb.org//movie/460059-burn-out
https://www.themoviedb.org//movie/12144-the-land-before-time
https://www.themoviedb.org//movie/547-the-horse-whisperer
https://www.themoviedb.org//movie/311667-manhattan-night
https://www.themoviedb.org//movie/50357-apollo-18
https://www.themoviedb.org//m

https://www.themoviedb.org//movie/572751-seized
https://www.themoviedb.org//movie/97038-the-rose-seller
https://www.themoviedb.org//movie/9538-scanners


page number 315
https://www.themoviedb.org//movie/10833-heartbreakers
https://www.themoviedb.org//movie/9292-hudson-hawk
https://www.themoviedb.org//movie/7520-cocktail
https://www.themoviedb.org//movie/2665-airplane-ii-the-sequel
https://www.themoviedb.org//movie/70586-setup
https://www.themoviedb.org//movie/453278-the-rider
https://www.themoviedb.org//movie/336011-white-girl
https://www.themoviedb.org//movie/258480-carol
https://www.themoviedb.org//movie/2255-chasing-amy
https://www.themoviedb.org//movie/153104-wicked-minds
https://www.themoviedb.org//movie/164-breakfast-at-tiffany's
https://www.themoviedb.org//movie/8051-punch-drunk-love
https://www.themoviedb.org//movie/21575-a-prophet
https://www.themoviedb.org//movie/255798-burying-the-ex
https://www.themoviedb.org//movie/630766-a-night-of-horror-nightmare-radio
https://www.them



page number 322
https://www.themoviedb.org//movie/624808-love-wedding-repeat
https://www.themoviedb.org//movie/77875-playing-for-keeps
https://www.themoviedb.org//movie/11156-coco-before-chanel
https://www.themoviedb.org//movie/264525-the-perfect-dictatorship
https://www.themoviedb.org//movie/11137-the-prince-&-me
https://www.themoviedb.org//movie/5460-cruel-intentions-3
https://www.themoviedb.org//movie/8738-the-final-countdown
https://www.themoviedb.org//movie/662844-angels-fallen
https://www.themoviedb.org//movie/11185-see-no-evil,-hear-no-evil
https://www.themoviedb.org//movie/877793-ablaze
https://www.themoviedb.org//movie/25793-precious
https://www.themoviedb.org//movie/463684-velvet-buzzsaw
https://www.themoviedb.org//movie/414977-sam
https://www.themoviedb.org//movie/4995-boogie-nights
https://www.themoviedb.org//movie/543976-marvel-zombies-vs-army-of-darkness
https://www.themoviedb.org//movie/68179-escape-from-planet-earth
https://www.themoviedb.org//movie/9972-lock-up
https

https://www.themoviedb.org//movie/20789-the-flyboys
https://www.themoviedb.org//movie/791568-settlers
https://www.themoviedb.org//movie/264656-the-homesman


page number 329
https://www.themoviedb.org//movie/9208-broken-arrow
https://www.themoviedb.org//movie/9290-crocodile-dundee-in-los-angeles
https://www.themoviedb.org//movie/617786-the-other-lamb
https://www.themoviedb.org//movie/115929-cool-kids-don't-cry
https://www.themoviedb.org//movie/3001-how-to-steal-a-million
https://www.themoviedb.org//movie/355066-bad-cat
https://www.themoviedb.org//movie/512901-the-knight-of-shadows-between-yin-and-yang
https://www.themoviedb.org//movie/354859-the-promise
https://www.themoviedb.org//movie/290729-sniper-legacy
https://www.themoviedb.org//movie/2266-paris,-je-t'aime
https://www.themoviedb.org//movie/11400-the-new-world
https://www.themoviedb.org//movie/609408-tutta-un'altra-vita
https://www.themoviedb.org//movie/491926-resistance
https://www.themoviedb.org//movie/10648-magnum-force
https:/

...

https://www.themoviedb.org//movie/509080-lego-marvel-super-heroes-guardians-of-the-galaxy---the-thanos-threat
https://www.themoviedb.org//movie/23030-arabesque
https://www.themoviedb.org//movie/11563-the-young-master
https://www.themoviedb.org//movie/14028-behind-enemy-lines-iii-colombia
https://www.themoviedb.org//movie/59408-deep-end
https://www.themoviedb.org//movie/10616-contract-killer
https://www.themoviedb.org//movie/59387-the-powerpuff-girls-movie
https://www.themoviedb.org//movie/60670-the-crimson-rivers
https://www.themoviedb.org//movie/2291-jacob's-ladder
https://www.themoviedb.org//movie/8874-my-best-friend's-wedding
https://www.themoviedb.org//movie/534259-blinded-by-the-light
https://www.themoviedb.org//movie/252391-kiki's-delivery-service
https://www.themoviedb.org//movie/39862-crocodile-2-death-swamp
https://www.themoviedb.org//movie/33320-millennium-actress
https://www.themoviedb.org//movie/11868-dracula
https://www.themoviedb.org//movie/9922-the-postman


page number 

https://www.themoviedb.org//movie/499319-duck-butter
https://www.themoviedb.org//movie/107748-sin-tetas-no-hay-paraíso
https://www.themoviedb.org//movie/9298-ali-g-indahouse
https://www.themoviedb.org//movie/532-a-close-shave
https://www.themoviedb.org//movie/24126-warlock-the-armageddon
https://www.themoviedb.org//movie/488113-christmas-wedding-planner
https://www.themoviedb.org//movie/21078-tom-and-jerry-blast-off-to-mars!
https://www.themoviedb.org//movie/1934-shakespeare-in-love
https://www.themoviedb.org//movie/552938-cucuy-the-boogeyman
https://www.themoviedb.org//movie/505579-bent
https://www.themoviedb.org//movie/45317-the-fighter
https://www.themoviedb.org//movie/11040-little-big-man
https://www.themoviedb.org//movie/271714-love-&-mercy
https://www.themoviedb.org//movie/560704-beyond-white-space
https://www.themoviedb.org//movie/500840-i'm-thinking-of-ending-things
https://www.themoviedb.org//movie/295592-monkey-kingdom
https://www.themoviedb.org//movie/71677-assassination-gam

https://www.themoviedb.org//movie/10396-tequila-sunrise
https://www.themoviedb.org//movie/459947-jackals
https://www.themoviedb.org//movie/26602-summer-catch
https://www.themoviedb.org//movie/819799-plush-boom!
https://www.themoviedb.org//movie/203834-jessabelle
https://www.themoviedb.org//movie/455605-haunted-hospital-heilstatten
https://www.themoviedb.org//movie/2832-identity
https://www.themoviedb.org//movie/304357-woman-in-gold
https://www.themoviedb.org//movie/40170-mortal-kombat-rebirth
https://www.themoviedb.org//movie/9944-the-pelican-brief
https://www.themoviedb.org//movie/250480-the-many-adventures-of-winnie-the-pooh
https://www.themoviedb.org//movie/519010-pain-and-glory
https://www.themoviedb.org//movie/75862-the-great-ghost-rescue
https://www.themoviedb.org//movie/48669-terror-trap
https://www.themoviedb.org//movie/40161-the-decoy-bride
https://www.themoviedb.org//movie/13967-miss-potter
https://www.themoviedb.org//movie/9929-asterix-and-cleopatra


page number 386
https:/

https://www.themoviedb.org//movie/2778-clifford
https://www.themoviedb.org//movie/18441-death-of-a-ghost-hunter
https://www.themoviedb.org//movie/368031-friend-request
https://www.themoviedb.org//movie/86837-amour
https://www.themoviedb.org//movie/16314-3-ninjas
https://www.themoviedb.org//movie/41211-heartbreaker
https://www.themoviedb.org//movie/501395-true-history-of-the-kelly-gang
https://www.themoviedb.org//movie/429202-vox-lux
https://www.themoviedb.org//movie/566076-the-united-states-vs.-billie-holiday
https://www.themoviedb.org//movie/4255-the-party-at-kitty-and-stud's
https://www.themoviedb.org//movie/34750-raw-justice
https://www.themoviedb.org//movie/269650-appleseed-alpha
https://www.themoviedb.org//movie/25523-the-ghost-of-lord-farquaad
https://www.themoviedb.org//movie/81232-foster
https://www.themoviedb.org//movie/28942-the-brood
https://www.themoviedb.org//movie/2897-around-the-world-in-eighty-days
https://www.themoviedb.org//movie/401545-everybody-knows
https://www.the

https://www.themoviedb.org//movie/199890-un-dios-prohibido
https://www.themoviedb.org//movie/89455-360
https://www.themoviedb.org//movie/10403-the-player
https://www.themoviedb.org//movie/11916-two-mules-for-sister-sara
https://www.themoviedb.org//movie/448194-pyewacket
https://www.themoviedb.org//movie/47854-legend-of-the-fist-the-return-of-chen-zhen
https://www.themoviedb.org//movie/312051-mi-gente-linda,-mi-gente-bella
https://www.themoviedb.org//movie/11953-kagemusha
https://www.themoviedb.org//movie/2742-naked-lunch
https://www.themoviedb.org//movie/144-wings-of-desire
https://www.themoviedb.org//movie/209451-jersey-boys
https://www.themoviedb.org//movie/11584-roxanne
https://www.themoviedb.org//movie/471968-the-last-warrior
https://www.themoviedb.org//movie/11692-the-adventures-of-pluto-nash
https://www.themoviedb.org//movie/9407-red-corner


page number 400
https://www.themoviedb.org//movie/419706-mary
https://www.themoviedb.org//movie/132344-before-midnight
https://www.themovie

In [255]:
get_data(starting=401,ending=500)

page number 401
https://www.themoviedb.org//movie/62459-icarly-igo-to-japan
https://www.themoviedb.org//movie/14844-chasing-liberty
https://www.themoviedb.org//movie/10622-mr.-nice-guy
https://www.themoviedb.org//movie/84184-celeste-&-jesse-forever
https://www.themoviedb.org//movie/616803-breaking-surface
https://www.themoviedb.org//movie/11616-go-for-it
https://www.themoviedb.org//movie/506-marnie
https://www.themoviedb.org//movie/652483-airplane-mode
https://www.themoviedb.org//movie/611605-1br
https://www.themoviedb.org//movie/11076-fly-away-home
https://www.themoviedb.org//movie/14047-the-great-debaters
https://www.themoviedb.org//movie/308032-the-stanford-prison-experiment
https://www.themoviedb.org//movie/124470-exists
https://www.themoviedb.org//movie/13483-awake
https://www.themoviedb.org//movie/1245-the-remains-of-the-day
https://www.themoviedb.org//movie/12262-the-hills-have-eyes
https://www.themoviedb.org//movie/2841-a-very-long-engagement
https://www.themoviedb.org//movie/1



page number 408
https://www.themoviedb.org//movie/3053-the-fearless-vampire-killers
https://www.themoviedb.org//movie/770254-back-to-the-outback
https://www.themoviedb.org//movie/10188-the-sisterhood-of-the-traveling-pants-2
https://www.themoviedb.org//movie/874152-christmas-in-my-heart
https://www.themoviedb.org//movie/349067-tigers-are-not-afraid
https://www.themoviedb.org//movie/4965-impostor
https://www.themoviedb.org//movie/455043-god-of-war
https://www.themoviedb.org//movie/254263-the-swan-princess-a-royal-family-tale
https://www.themoviedb.org//movie/48466-scared-shrekless
https://www.themoviedb.org//movie/20771-kim-possible-so-the-drama
https://www.themoviedb.org//movie/118732-for-a-cop's-hide
https://www.themoviedb.org//movie/42949-arthur-3-the-war-of-the-two-worlds
https://www.themoviedb.org//movie/15616-private-valentine-blonde-&-dangerous
https://www.themoviedb.org//movie/295315-sex-ed
https://www.themoviedb.org//movie/525657-daughter-of-the-wolf
https://www.themoviedb.or

https://www.themoviedb.org//movie/167-k-pax
https://www.themoviedb.org//movie/51209-kill-the-irishman
https://www.themoviedb.org//movie/53064-midori


page number 415
https://www.themoviedb.org//movie/675493-two-yellow-lines
https://www.themoviedb.org//movie/470610-endless
https://www.themoviedb.org//movie/160055-relations---the-love-story-from-denmark
https://www.themoviedb.org//movie/468198-the-place
https://www.themoviedb.org//movie/372782-manhunt
https://www.themoviedb.org//movie/10956-joe-dirt
https://www.themoviedb.org//movie/9557-boat-trip
https://www.themoviedb.org//movie/318781-colonia
https://www.themoviedb.org//movie/274109-palm-trees-in-the-snow
https://www.themoviedb.org//movie/26263-death-wish-4-the-crackdown
https://www.themoviedb.org//movie/850056-disney-princess-remixed-an-ultimate-princess-celebration
https://www.themoviedb.org//movie/578764-a-ciegas
https://www.themoviedb.org//movie/352885-the-book-of-love
https://www.themoviedb.org//movie/3902-i'm-not-there
https://

https://www.themoviedb.org//movie/589612-a-call-to-spy
https://www.themoviedb.org//movie/102207-houba!-on-the-trail-of-the-marsupilami


page number 422
https://www.themoviedb.org//movie/14637-nothing-but-the-truth
https://www.themoviedb.org//movie/9501-crying-freeman
https://www.themoviedb.org//movie/121826-quartet
https://www.themoviedb.org//movie/382501-eternity
https://www.themoviedb.org//movie/10207-message-in-a-bottle
https://www.themoviedb.org//movie/768520-a-california-christmas
https://www.themoviedb.org//movie/647325-the-thing-about-harry
https://www.themoviedb.org//movie/84305-lay-the-favorite
https://www.themoviedb.org//movie/27740-a-return-to-salem's-lot
https://www.themoviedb.org//movie/13341-bad-news-bears
https://www.themoviedb.org//movie/282848-doctor-who-the-time-of-the-doctor
https://www.themoviedb.org//movie/88273-a-royal-affair
https://www.themoviedb.org//movie/11004-wonder-boys
https://www.themoviedb.org//movie/25329-the-echo
https://www.themoviedb.org//movie/5402

https://www.themoviedb.org//movie/19904-the-stepfather
https://www.themoviedb.org//movie/558095-all-is-true
https://www.themoviedb.org//movie/844636-jungle-run
https://www.themoviedb.org//movie/845-strangers-on-a-train
https://www.themoviedb.org//movie/691582-a-celebration-of-the-music-from-coco


page number 429
https://www.themoviedb.org//movie/664300-shiva-baby
https://www.themoviedb.org//movie/363093-the-tiger-an-old-hunter's-tale
https://www.themoviedb.org//movie/340022-gringo
https://www.themoviedb.org//movie/17814-assault-on-precinct-13
https://www.themoviedb.org//movie/218784-freaks-of-nature
https://www.themoviedb.org//movie/196750-ghost-in-the-shell-arise---border-1-ghost-pain
https://www.themoviedb.org//movie/56590-all-star-superman
https://www.themoviedb.org//movie/325346-the-hollars
https://www.themoviedb.org//movie/1641-forces-of-nature
https://www.themoviedb.org//movie/15648-the-perfect-man
https://www.themoviedb.org//movie/9503-pope-joan
https://www.themoviedb.org//movi

...

https://www.themoviedb.org//movie/15590-the-prince-&-me-2-the-royal-wedding
https://www.themoviedb.org//movie/363579-the-age-of-shadows
https://www.themoviedb.org//movie/34205-halloweentown-ii-kalabar's-revenge
https://www.themoviedb.org//movie/463053-ibiza
https://www.themoviedb.org//movie/6691-priceless


page number 471
https://www.themoviedb.org//movie/12079-spun
https://www.themoviedb.org//movie/421560-giant
https://www.themoviedb.org//movie/9708-the-wicker-man
https://www.themoviedb.org//movie/433244-clinical
https://www.themoviedb.org//movie/438553-haruta-&-chika
https://www.themoviedb.org//movie/566451-rezero--starting-life-in-another-world--the-frozen-bond
https://www.themoviedb.org//movie/280874-killing-daddy
https://www.themoviedb.org//movie/9644-national-lampoon's-loaded-weapon-1
https://www.themoviedb.org//movie/892910-the-dead-collectors
https://www.themoviedb.org//movie/436452-treasures
https://www.themoviedb.org//movie/24478-beethoven's-4th
https://www.themoviedb.org//m

https://www.themoviedb.org//movie/38357-morning-glory
https://www.themoviedb.org//movie/46581-immoral-tales
https://www.themoviedb.org//movie/520360-king-of-thieves
https://www.themoviedb.org//movie/505026-death-on-the-nile
https://www.themoviedb.org//movie/362058-the-snow-queen-3-fire-and-ice
https://www.themoviedb.org//movie/25189-the-mirror-has-two-faces


page number 478
https://www.themoviedb.org//movie/14043-nancy-drew
https://www.themoviedb.org//movie/471608-eat,-play,-love
https://www.themoviedb.org//movie/284296-top-five
https://www.themoviedb.org//movie/40728-air-the-motion-picture
https://www.themoviedb.org//movie/55420-another-earth
https://www.themoviedb.org//movie/567733-crypto
https://www.themoviedb.org//movie/14247-sniper-3
https://www.themoviedb.org//movie/9978-stormbreaker
https://www.themoviedb.org//movie/301348-elvis-&-nixon
https://www.themoviedb.org//movie/273271-time-lapse
https://www.themoviedb.org//movie/259954-5-to-7
https://www.themoviedb.org//movie/50850-the

https://www.themoviedb.org//movie/522039-the-last-black-man-in-san-francisco
https://www.themoviedb.org//movie/520016-driven
https://www.themoviedb.org//movie/429467-the-lodgers


page number 485
https://www.themoviedb.org//movie/575138-the-new-king-of-comedy
https://www.themoviedb.org//movie/14749-space-buddies
https://www.themoviedb.org//movie/381707-our-lovers
https://www.themoviedb.org//movie/13503-texas-rangers
https://www.themoviedb.org//movie/3085-his-girl-friday
https://www.themoviedb.org//movie/523371-patients-of-a-saint
https://www.themoviedb.org//movie/467-the-hole
https://www.themoviedb.org//movie/646150-acceleration
https://www.themoviedb.org//movie/633553-fútbol-club-maradona
https://www.themoviedb.org//movie/666390-bunty-aur-babli-2
https://www.themoviedb.org//movie/15088-the-cutting-edge-chasing-the-dream
https://www.themoviedb.org//movie/5680-elvira,-mistress-of-the-dark
https://www.themoviedb.org//movie/674996-the-king-of-all-the-world
https://www.themoviedb.org//movi

https://www.themoviedb.org//movie/27902-love-wrecked
https://www.themoviedb.org//movie/603519-beats
https://www.themoviedb.org//movie/300155-a-royal-night-out
https://www.themoviedb.org//movie/606679-the-high-note
https://www.themoviedb.org//movie/583116-the-house-next-door-meet-the-blacks-2


page number 492
https://www.themoviedb.org//movie/324562-the-tribes-of-palos-verdes
https://www.themoviedb.org//movie/466876-gnome-alone
https://www.themoviedb.org//movie/9613-spider
https://www.themoviedb.org//movie/84175-beasts-of-the-southern-wild
https://www.themoviedb.org//movie/9051-10
https://www.themoviedb.org//movie/653723-boys-state
https://www.themoviedb.org//movie/13042-presto
https://www.themoviedb.org//movie/13531-empire-records
https://www.themoviedb.org//movie/10774-network
https://www.themoviedb.org//movie/152599-the-immigrant
https://www.themoviedb.org//movie/9715-hope-floats
https://www.themoviedb.org//movie/571650-paradise-beach
https://www.themoviedb.org//movie/527632-hope-sp

https://www.themoviedb.org//movie/328869-meadowland
https://www.themoviedb.org//movie/157354-fruitvale-station
https://www.themoviedb.org//movie/17612-lost-and-delirious
https://www.themoviedb.org//movie/152795-the-congress
https://www.themoviedb.org//movie/613096-i-love-you,-stupid
https://www.themoviedb.org//movie/513857-the-nanny


page number 499
https://www.themoviedb.org//movie/683665-007-licence-to-restore
https://www.themoviedb.org//movie/17182-eye-for-an-eye
https://www.themoviedb.org//movie/15037-can't-hardly-wait
https://www.themoviedb.org//movie/10711-french-connection-ii
https://www.themoviedb.org//movie/4240-titanic-2000
https://www.themoviedb.org//movie/21935-the-haunting-hour-don't-think-about-it
https://www.themoviedb.org//movie/364733-blind
https://www.themoviedb.org//movie/14245-sniper-2
https://www.themoviedb.org//movie/26505-arthur-and-the-revenge-of-maltazard
https://www.themoviedb.org//movie/10843-after-hours
https://www.themoviedb.org//movie/606-out-of-africa
ht

In [256]:
len(movie_data_list)

10000

In [258]:
keys = movie_data_list[0].keys()

In [259]:
keys

dict_keys(['budget', 'revenue', 'content_score', 'user_score', 'genres', 'director', 'release_year', 'release_month', 'release_day', 'release_weekday', 'run_time (in minuts)', 'title', 'url'])

In [260]:
with open('movies.csv','w',newline='') as data_input:
    data_input = csv.DictWriter(data_input,keys)
    data_input.writeheader()
    data_input.writerows(movie_data_list)

In [261]:
df = pd.read_csv('movies.csv')

In [262]:
df

,budget,revenue,content_score,user_score,genres,director,release_year,release_month,release_day,release_weekday,run_time (in minuts),title,url
0,150000000.0,430238384.0,100.0,79.0,"Action,Adventure,Fantasy",Destin Daniel Cretton,2021.0,9.0,3.0,4.0,132.0,Shang-Chi and the Legend of the Ten Rings,https://www.themoviedb.org//movie/566525-shang...
1,242000000.0,708000000.0,100.0,76.0,"Adventure,Action,Thriller",Cary Joji Fukunaga,2021.0,9.0,30.0,3.0,163.0,No Time to Die,https://www.themoviedb.org//movie/370172-no-ti...
2,110000000.0,424000000.0,100.0,68.0,"Science Fiction,Action,Adventure",Kelly Marcel,2021.0,10.0,1.0,4.0,97.0,Venom: Let There Be Carnage,https://www.themoviedb.org//movie/580489-venom...
3,NaN,NaN,100.0,82.0,"Science Fiction,Drama,Adventure",Miguel Sapochnik,2021.0,11.0,4.0,3.0,115.0,Finch,https://www.themoviedb.org//movie/522402-finch
4,NaN,NaN,100.0,57.0,"Action,Thriller,Science Fiction",Edward Drake,2021.0,12.0,16.0,3.0,108.0,Apex,https://www.themoviedb.org//movie/763164-apex
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,100.0,82.0,"Adventure,Drama,Family",Mark Drury Taylor,2019.0,12.0,6.0,4.0,95.0,Saving Flora,https://www.themoviedb.org//movie/570488-savin...
9996,19000000.0,272742922.0,100.0,83.0,"Crime,Drama,Thriller",Jonathan Demme,1991.0,2.0,14.0,3.0,119.0,The Silence of the Lambs,https://www.themoviedb.org//movie/274-the-sile...
9997,NaN,NaN,100.0,58.0,"TV Movie,Fantasy,Comedy,Romance,Family",Gil Junger,2010.0,12.0,12.0,6.0,85.0,Christmas Cupid,https://www.themoviedb.org//movie/52060-christ...
9998,NaN,NaN,100.0,64.0,"Drama,TV Movie",Barry Levinson,2018.0,4.0,7.0,5.0,105.0,Paterno,https://www.themoviedb.org//movie/467867-paterno
